In [1]:
import numpy as np
import pandas as pd
import torch
import sys
code_path = '/home/user-home/yujie/0_PBCNetv2/0_PBCNET'
sys.path.append(f'{code_path}/model_code')
from torch.utils.data import DataLoader
from tqdm import tqdm
from utilis.utilis import pkl_load

In [2]:
def minmax(tensor_x):
    min_val = torch.min(tensor_x)
    max_val = torch.max(tensor_x)
    tensor_x_normalized = (tensor_x - min_val) / (max_val - min_val)

    return tensor_x_normalized

In [3]:
model = torch.load(f"{code_path}/PBCNet2.pth", map_location=torch.device('cpu'), weights_only=False)

In [4]:
sys = f'{code_path}/data/F-Opt/pose/5G4O_1_bad.pkl'
g = pkl_load(sys)
index = np.where(g.nodes['atom'].data['x'] == 9)[0][0]
index

1

In [5]:
g.nodes['atom'].data['pos'][index]

tensor([ 92.9613,  95.5270, -40.8474])

In [6]:
def mean_sca(att, type):
    input = torch.zeros(torch.max(type)+1)
    v = input.scatter_reduce_(0, type, att, reduce="mean", include_self=False)
    return v

In [7]:

g.edges['int'].data['bond_scalar'] = torch.clip(g.edges['int'].data['bond_scalar'].long(), min=0, max=4)
g.edges['ind'].data['bond_scalar'] = torch.clip(g.edges['ind'].data['bond_scalar'].long(), min=0, max=4)
emb,att = model._readout(g, bb=False)
att =att[0]
bond_type = 'int'
print(att.shape)
att1 = att.reshape(att.shape[0], -1)

att1_ = torch.mean(att1,dim=-1)
mask = g.nodes['atom'].data['type'].detach().numpy()

def src_norm_dst(src_field, dst_field, out_field):
    def func(edges):
        return {out_field: torch.sqrt(torch.sum((edges.src[src_field] - edges.dst[dst_field]) ** 2, dim=1))}
    return func
g.apply_edges(src_norm_dst('pos', 'pos', 'dist'), etype=bond_type)
d = g.edges[bond_type].data['dist']
type = g.edges[bond_type].data['bond_scalar'].long()
a = mean_sca(att1_, type)
print(a)

torch.Size([1686, 96, 3])
tensor([0.0097, 0.0090, 0.0000, 0.0146, 0.0189],
       grad_fn=<ScatterReduceBackward0>)


In [8]:
g = pkl_load(sys)
a = np.intersect1d(np.where(g.edges['int'].data['bond_scalar'].long() == 6)[0], np.where(g.edges(etype = 'int')[1].detach().numpy() == index)[0])
b = np.intersect1d(np.where(g.edges['int'].data['bond_scalar'].long() == 6)[0], np.where(g.edges(etype = 'int')[0].detach().numpy() == index)[0])
minmax(att1_[b])

tensor([0.8172, 0.5047, 1.0000, 0.5073, 0.6599, 0.8835, 0.5649, 0.4010, 0.6711,
        0.7415, 0.8811, 0.1205, 0.8651, 0.0000, 0.3338, 0.8173, 0.9853, 0.8651,
        0.5415, 0.9248], grad_fn=<DivBackward0>)

In [ ]:
#pdbcnet:
[0.6793, 0.9124, 0.0000, 0.6731, 0.3564, 1.0000, 0.9430, 0.6803, 0.6012,
0.9481, 0.2420, 0.7375, 0.3170, 0.6041, 0.5990, 0.6631, 0.1871, 0.7009,
0.5912, 0.8434]

# key atoms coor : [ 93.8250,  94.3800, -37.6840]   [ 95.9460,  96.5480, -39.4560]

In [9]:
g.nodes['atom'].data['pos'][g.edges(etype = 'int')[0].detach().numpy()[a]]

tensor([[ 94.0090,  93.0250, -36.9980],
        [ 93.8250,  94.3800, -37.6840],
        [ 92.7220,  94.6830, -38.1420],
        [ 93.7040,  91.8870, -37.9970],
        [ 94.9100,  95.1620, -37.7320],
        [ 94.9430,  96.5040, -38.3020],
        [ 95.9460,  96.5480, -39.4560],
        [ 97.0410,  96.0030, -39.3310],
        [ 95.3610,  97.5030, -37.2080],
        [ 95.5590,  97.2460, -40.5310],
        [ 96.4000,  97.5840, -41.6780],
        [ 96.0490,  99.0030, -42.1550],
        [ 94.9000,  99.4310, -42.0320],
        [ 96.2220,  96.6000, -42.8750],
        [ 96.9470,  95.2640, -42.6520],
        [ 94.7660,  96.3830, -43.3140],
        [ 90.1970,  98.9490, -38.9140],
        [ 89.5070,  96.3780, -37.8740],
        [ 89.3880,  93.9960, -38.3380],
        [ 89.1210,  92.4580, -40.7080]])

In [10]:
minmax(att1_[b])[g.nodes['atom'].data['x'][g.edges(etype = 'int')[0].detach().numpy()[a]] == 6]

tensor([0.8172, 0.5047, 0.5073, 0.8835, 0.5649, 0.6711, 0.8811, 0.1205, 0.0000,
        0.3338, 0.8173, 0.8651], grad_fn=<IndexBackward0>)

In [11]:
torch.mean(minmax(att1_[b])[g.nodes['atom'].data['x'][g.edges(etype = 'int')[0].detach().numpy()[a]] == 6])

tensor(0.5805, grad_fn=<MeanBackward0>)